# T1187 Forced Authentication

-----------------------------------------------------------------------

## Technique Description

Adversaries may gather credential material by invoking or forcing a user to automatically provide authentication information through a mechanism in which they can intercept.

The Server Message Block (SMB) protocol is commonly used in Windows networks for authentication and communication between systems for access to resources and file sharing. When a Windows system attempts to connect to an SMB resource it will automatically attempt to authenticate and send credential information for the current user to the remote system. (Citation: Wikipedia Server Message Block) This behavior is typical in enterprise environments so that users do not need to enter credentials to access network resources.

Web Distributed Authoring and Versioning (WebDAV) is also typically used by Windows systems as a backup protocol when SMB is blocked or fails. WebDAV is an extension of HTTP and will typically operate over TCP ports 80 and 443. (Citation: Didier Stevens WebDAV Traffic) (Citation: Microsoft Managing WebDAV Security)

Adversaries may take advantage of this behavior to gain access to user account hashes through forced SMB/WebDAV authentication. An adversary can send an attachment to a user through spearphishing that contains a resource link to an external server controlled by the adversary (i.e. [Template Injection](https://attack.mitre.org/techniques/T1221)), or place a specially crafted file on navigation path for privileged accounts (e.g. .SCF file placed on desktop) or on a publicly accessible share to be accessed by victim(s). When the user's system accesses the untrusted resource it will attempt authentication and send information, including the user's hashed credentials, over SMB to the adversary controlled server. (Citation: GitHub Hashjacking) With access to the credential hash, an adversary can perform off-line [Brute Force](https://attack.mitre.org/techniques/T1110) cracking to gain access to plaintext credentials. (Citation: Cylance Redirect to SMB)

There are several different ways this can occur. (Citation: Osanda Stealing NetNTLM Hashes) Some specifics from in-the-wild use include:

* A spearphishing attachment containing a document with a resource that is automatically loaded when the document is opened (i.e. [Template Injection](https://attack.mitre.org/techniques/T1221)). The document can include, for example, a request similar to <code>file[:]//[remote address]/Normal.dotm</code> to trigger the SMB request. (Citation: US-CERT APT Energy Oct 2017)
* A modified .LNK or .SCF file with the icon filename pointing to an external reference such as <code>\\[remote address]\pic.png</code> that will force the system to load the resource when the icon is rendered to repeatedly gather credentials. (Citation: US-CERT APT Energy Oct 2017)

## Technique Detection

Monitor for SMB traffic on TCP ports 139, 445 and UDP port 137 and WebDAV traffic attempting to exit the network to unknown external systems. If attempts are detected, then investigate endpoint data sources to find the root cause. For internal traffic, monitor the workstation-to-workstation unusual (vs. baseline) SMB traffic. For many networks there should not be any, but it depends on how systems on the network are configured and where resources are located.

Monitor creation and modification of .LNK, .SCF, or any other files on systems and within virtual environments that contain resources that point to external network resources as these could be used to gather credentials when the files are rendered. (Citation: US-CERT APT Energy Oct 2017)

-----------------------------------------------------------------------

### Tactics:

  *   Credential-Access

### Platforms:

  * Windows

### Adversary Required Permissions:

  * User

### Data Sources:

  * **File:** File Access

  * **Network Traffic:** Network Traffic Flow

  * **File:** File Modification

  * **File:** File Creation

  * **Network Traffic:** Network Traffic Content

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| DarkHydrus | [DarkHydrus](https://attack.mitre.org/groups/G0079) used [Template Injection](https://attack.mitre.org/techniques/T1221) to launch an authentication window for users to enter their credentials.(Citation: Unit 42 Phishery Aug 2018)| 
| Dragonfly | [Dragonfly](https://attack.mitre.org/groups/G0035) has gathered hashed user credentials over SMB using spearphishing attachments with external resource links and by modifying .LNK file icon resources to collect credentials from virtualized systems.(Citation: US-CERT TA18-074A)(Citation: Gigamon Berserk Bear October 2021)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1187)

  * [Wikipedia Server Message Block](https://en.wikipedia.org/wiki/Server_Message_Block), Wikipedia. (2017, December 16). Server Message Block. Retrieved December 21, 2017.

  * [Didier Stevens Webdav Traffic](https://blog.didierstevens.com/2017/11/13/webdav-traffic-to-malicious-sites/), Stevens, D. (2017, November 13). WebDAV Traffic To Malicious Sites. Retrieved December 21, 2017.

  * [Microsoft Managing Webdav Security](https://www.microsoft.com/technet/prodtechnol/WindowsServer2003/Library/IIS/4beddb35-0cba-424c-8b9b-a5832ad8e208.mspx), Microsoft. (n.d.). Managing WebDAV Security (IIS 6.0). Retrieved December 21, 2017.

  * [Github Hashjacking](https://github.com/hob0/hashjacking), Dunning, J. (2016, August 1). Hashjacking. Retrieved December 21, 2017.

  * [Cylance Redirect To Smb](https://www.cylance.com/content/dam/cylance/pdfs/white_papers/RedirectToSMB.pdf), Cylance. (2015, April 13). Redirect to SMB. Retrieved December 21, 2017.

  * [Osanda Stealing Netntlm Hashes](https://osandamalith.com/2017/03/24/places-of-interest-in-stealing-netntlm-hashes/), Osanda Malith Jayathissa. (2017, March 24). Places of Interest in Stealing NetNTLM Hashes. Retrieved January 26, 2018.

  * [Us-Cert Apt Energy Oct 2017](https://www.us-cert.gov/ncas/alerts/TA17-293A), US-CERT. (2017, October 20). Alert (TA17-293A): Advanced Persistent Threat Activity Targeting Energy and Other Critical Infrastructure Sectors. Retrieved November 2, 2017.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Information Here

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- Information Here

## Analytical References

  * [CISA Alerts TA18-074A (cisa)](https://us-cert.cisa.gov/ncas/alerts/TA18-074A)
  * [Elastic Prebuilt Rules](https://www.elastic.co/guide/en/siem/guide/current/prebuilt-rules.html)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Prioritize WebDav traffic going outbound. Many times internal WebDav protocol is due to a NAS.

#### Analytic 1

  * **Information:** Identifies suspicious child processes of frequently targeted Microsoft Office applications (Word, PowerPoint, Excel). These child processes are often launched during exploitation of Office applications or from documents with malicious macros.

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:**  This is a framework query meant to be tailored.

  * **Query:** ```event.action:"Process Create (rule: ProcessCreate)" and
process.parent.name:(eqnedt32.exe or excel.exe or fltldr.exe or
msaccess.exe or mspub.exe or powerpnt.exe or winword.exe) and
process.name:(Microsoft.Workflow.Compiler.exe or arp.exe or
atbroker.exe or bginfo.exe or bitsadmin.exe or cdb.exe or certutil.exe
or cmd.exe or cmstp.exe or cscript.exe or csi.exe or dnx.exe or
dsget.exe or dsquery.exe or forfiles.exe or fsi.exe or ftp.exe or
gpresult.exe or hostname.exe or ieexec.exe or iexpress.exe or
installutil.exe or ipconfig.exe or mshta.exe or msxsl.exe or
nbtstat.exe or net.exe or net1.exe or netsh.exe or netstat.exe or
nltest.exe or odbcconf.exe or ping.exe or powershell.exe or pwsh.exe
or qprocess.exe or quser.exe or qwinsta.exe or rcsi.exe or reg.exe or
regasm.exe or regsvcs.exe or regsvr32.exe or sc.exe or schtasks.exe or
systeminfo.exe or tasklist.exe or tracert.exe or whoami.exe or
wmic.exe or wscript.exe or xwizard.exe)```

#### Analytic 2

  * **Information:** Identifies suspicious child processes of Microsoft Outlook. These child processes are often associated with spear phishing activity. 

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:** This is a framework query meant to be tailored.

  * **Query:** ```event.action:"Process Create (rule: ProcessCreate)" and
process.parent.name:outlook.exe and
process.name:(Microsoft.Workflow.Compiler.exe or arp.exe or
atbroker.exe or bginfo.exe or bitsadmin.exe or cdb.exe or certutil.exe
or cmd.exe or cmstp.exe or cscript.exe or csi.exe or dnx.exe or
dsget.exe or dsquery.exe or forfiles.exe or fsi.exe or ftp.exe or
gpresult.exe or hostname.exe or ieexec.exe or iexpress.exe or
installutil.exe or ipconfig.exe or mshta.exe or msxsl.exe or
nbtstat.exe or net.exe or net1.exe or netsh.exe or netstat.exe or
nltest.exe or odbcconf.exe or ping.exe or powershell.exe or pwsh.exe
or qprocess.exe or quser.exe or qwinsta.exe or rcsi.exe or reg.exe or
regasm.exe or regsvcs.exe or regsvr32.exe or sc.exe or schtasks.exe or
systeminfo.exe or tasklist.exe or tracert.exe or whoami.exe or
wmic.exe or wscript.exe or xwizard.exe)```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

